In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('darkgrid')
plt.style.use('dark_background')

In [2]:
data = pd.read_csv("train_motion_data.csv")
data.head()

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,NORMAL,3581629
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,NORMAL,3581630
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,NORMAL,3581630
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,NORMAL,3581631
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,NORMAL,3581631


In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

encoder = LabelEncoder()

data['Class'] = encoder.fit_transform(data['Class'])
y = to_categorical(data['Class'])

In [4]:
scaler = StandardScaler()
features = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']
data[features] = scaler.fit_transform(data[features])

In [5]:
from sklearn.model_selection import train_test_split

# Define sequence length (e.g., 10 previous timesteps per sample)
sequence_length = 10

def create_sequences(data, labels, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(labels[i + seq_length])
    return np.array(X), np.array(y)

# Convert features and labels into sequences
X_data, y_data = create_sequences(data[features].values, y, sequence_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print("Training shape:", X_train.shape, "Testing shape:", X_test.shape)
# Output Example: (samples, timesteps=10, features=6)
# NOTE: modify timesteps to test accuracy

Training shape: (2907, 10, 6) Testing shape: (727, 10, 6)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [7]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(sequence_length, len(features))), 
    Dropout(0.2),
    LSTM(32), 
    Dropout(0.2),
    Dense(16, activation='relu'), 
    Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

2025-02-14 22:20:07.279342: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-02-14 22:20:07.279589: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-02-14 22:20:07.279613: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-02-14 22:20:07.280008: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-14 22:20:07.280445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            18176     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 3)                 51        
                                                                 
Total params: 31171 (121.76 KB)
Trainable params: 31171 

In [8]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50


2025-02-14 22:20:20.945310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:21.171355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:21.765665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:23.438918: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:24.048913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - ETA: 0s - loss: 1.0688 - accuracy: 0.4145

2025-02-14 22:20:26.406753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:26.462808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:20:26.501104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - 7s 28ms/step - loss: 1.0688 - accuracy: 0.4145 - val_loss: 1.0615 - val_accuracy: 0.4292
Epoch 2/50
91/91 [==============================] - 1s 16ms/step - loss: 0.9782 - accuracy: 0.4950 - val_loss: 0.9537 - val_accuracy: 0.5131
Epoch 3/50
91/91 [==============================] - 1s 15ms/step - loss: 0.8750 - accuracy: 0.5415 - val_loss: 0.8562 - val_accuracy: 0.5681
Epoch 4/50
91/91 [==============================] - 1s 15ms/step - loss: 0.8133 - accuracy: 0.5896 - val_loss: 0.8090 - val_accuracy: 0.5915
Epoch 5/50
91/91 [==============================] - 1s 15ms/step - loss: 0.7797 - accuracy: 0.6061 - val_loss: 0.7960 - val_accuracy: 0.5846
Epoch 6/50
91/91 [==============================] - 1s 15ms/step - loss: 0.7300 - accuracy: 0.6371 - val_loss: 0.7695 - val_accuracy: 0.6149
Epoch 7/50
91/91 [==============================] - 1s 15ms/step - loss: 0.6876 - accuracy: 0.6543 - val_loss: 0.7363 - val_accuracy: 0.6424
Epoch 8/50
91/91 [======

In [9]:
y_pred = model.predict(X_test)

# Convert predictions back to label format
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Decode labels back to original categories
y_pred_classes = encoder.inverse_transform(y_pred_labels)
y_test_classes = encoder.inverse_transform(y_test_labels)

 1/23 [>.............................] - ETA: 7s

2025-02-14 22:22:44.185761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:22:44.247078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-02-14 22:22:44.298753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 4ms/step


In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

  AGGRESSIVE       0.96      0.93      0.94       211
      NORMAL       0.92      0.91      0.91       260
        SLOW       0.92      0.94      0.93       256

    accuracy                           0.93       727
   macro avg       0.93      0.93      0.93       727
weighted avg       0.93      0.93      0.93       727



In [15]:
from tensorflow.keras.models import load_model

# Save the trained LSTM model
model.save('lstm_driving_behavior.h5')

/Users/aswatt/miniforge3/envs/BattOPT/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
import joblib

# Assuming 'scaler' was used for data normalization during training
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']